"""
E91 Protocol Simulation with Qiskit

This notebook demonstrates the Ekert 91 (E91) quantum key distribution protocol
using Qiskit. E91 leverages entanglement and Bell's theorem (specifically the
CHSH inequality) to establish a shared secret key between two parties (Alice
and Bob) and detect potential eavesdropping (Eve).
"""


## 1. Introduction to E91

The E91 protocol works as follows:
1.  **Source**: A source produces entangled qubit pairs (e.g., Bell states), sending one qubit to Alice and the other to Bob. We will use the $|\Psi^-\rangle = \frac{1}{\sqrt{2}}(|01\rangle - |0\rangle)$ state.
2.  **Measurement Basis Choice**: Alice and Bob independently and randomly choose measurement bases for each qubit they receive.
    * Alice's bases (angles relative to Z-axis): $A_1 = 0^\circ$ (Z), $A_2 = 45^\circ$, $A_3 = 90^\circ$ (X).
    * Bob's bases (angles relative to Z-axis): $B_1 = 45^\circ$, $B_2 = 90^\circ$ (X), $B_3 = 135^\circ$.
3.  **Measurement**: They measure their qubits in the chosen bases and record the outcomes (0 or 1).
4.  **Basis Reconciliation (Public Discussion)**: Alice and Bob publicly announce the sequence of bases they used for each measurement, but *not* the results.
5.  **Data Sifting & CHSH Test**: They use measurements where their bases correspond to specific pairs needed for the CHSH test (e.g., $(A_1, B_1)$, $(A_1, B_3)$, $(A_3, B_1)$, $(A_3, B_3)$). They calculate the CHSH correlation value $S$. In a noise-free, secure scenario, $|S| = 2\sqrt{2} \approx 2.828$. If $|S| \le 2$, it indicates either significant noise or the presence of an eavesdropper (classical limit).
6.  **Key Generation**: If the CHSH test indicates security ($|S|$ is sufficiently large, close to $2\sqrt{2}$), they use the measurement outcomes from the *other* basis combinations (e.g., $(A_1, B_2)$, $(A_2, B_1)$, $(A_2, B_2)$, etc.) to form their raw secret keys. For pairs where measurements are anti-correlated (like $A_2, B_1$), one party (e.g., Bob) flips their bits.
7.  **Post-Processing**: Classical error correction and privacy amplification are typically applied to the raw key (not implemented in this basic simulation).

## 2. Setup


Import necessary libraries and set simulation parameters.

In [1]:
# Import Qiskit and other useful libraries
import numpy as np
import math
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator # Use AerSimulator for simulation

In [2]:
# Parameters
NUM_QUBITS = 2 # E91 uses pairs of qubits
N_SHOTS = 2000 # Number of entangled pairs to generate and measure (adjust as needed)

In [3]:
# Define measurement bases angles (in radians)
# Alice's angles
alice_angles = {
    1: 0,          # A1: Z-basis (0 degrees)
    2: math.pi / 4, # A2: 45 degrees
    3: math.pi / 2  # A3: X-basis (90 degrees)
}
# Bob's angles
bob_angles = {
    1: math.pi / 4,   # B1: 45 degrees
    2: math.pi / 2,   # B2: X-basis (90 degrees)
    3: 3 * math.pi / 4 # B3: 135 degrees
}


In [4]:
# Basis choices for CHSH calculation (Ekert's original paper)
CHSH_BASIS_PAIRS = [(1, 1), (1, 3), (3, 1), (3, 3)] # (Alice_index, Bob_index)

# Mapping measurement outcomes {0, 1} to values {+1, -1} for correlation calculation
def outcome_to_value(outcome):
    return 1 if outcome == '0' else -1

print("Setup Complete.")
print(f"Simulating {N_SHOTS} rounds of E91.")
print(f"Alice's bases angles (rad): {alice_angles}")
print(f"Bob's bases angles (rad): {bob_angles}")
print(f"Basis pairs for CHSH test: {CHSH_BASIS_PAIRS}")

Setup Complete.
Simulating 2000 rounds of E91.
Alice's bases angles (rad): {1: 0, 2: 0.7853981633974483, 3: 1.5707963267948966}
Bob's bases angles (rad): {1: 0.7853981633974483, 2: 1.5707963267948966, 3: 2.356194490192345}
Basis pairs for CHSH test: [(1, 1), (1, 3), (3, 1), (3, 3)]


## 3. Quantum Simulation

We will simulate the process of generating entangled pairs, distributing them, choosing measurement bases, and measuring.

In [5]:
# Initialize simulator
# We use statevector simulator for ideal simulation, can be changed to others like 'qasm_simulator'
simulator = AerSimulator(method='statevector') # Use statevector simulator for perfect results initially

# Store results
results = [] # List to store (alice_basis, bob_basis, alice_outcome, bob_outcome) for each shot

In [6]:
# --- Function to create the Bell state |Psi-> ---
def create_bell_psi_minus():
    qr = QuantumRegister(NUM_QUBITS, 'q')
    cr = ClassicalRegister(NUM_QUBITS, 'c')
    qc = QuantumCircuit(qr, cr, name='Bell_Psi_Minus')
    qc.h(qr[0])      # Apply Hadamard to the first qubit
    qc.cx(qr[0], qr[1]) # Apply CNOT gate
    # To get |Psi-> from |Phi+> = 1/sqrt(2)(|00> + |11>), apply Z to q0 and X to q1
    qc.z(qr[0])
    qc.x(qr[1])
    qc.barrier() # Visual separator
    return qc, qr, cr


#### Function to apply measurement rotation
Measurement in a basis defined by angle 'theta' relative to Z-axis is achieved by rotating the state by Ry(-theta) before Z-measurement.

In [7]:
def apply_measurement_rotation(qc, qubit, basis_angle):
    # Note: Qiskit's measure operation is always in the Z-basis.
    # To measure in a basis rotated by theta (in X-Z plane), we apply Ry(-theta).
    if not np.isclose(basis_angle, 0): # No rotation needed for Z-basis (theta=0)
         # We apply Ry(angle) to measure in basis defined by angle -angle/2 ? No.
         # Ry(beta) rotates around Y. To measure along axis 'theta' from Z towards X,
         # we rotate the state vector by -theta around Y.
         qc.ry(-basis_angle, qubit) # Correct rotation to align measurement axis


In [9]:
# --- Simulation Loop ---
print("Starting simulation loop...")
for i in range(N_SHOTS):
    # 1. Alice and Bob randomly choose their bases
    alice_basis_choice = np.random.randint(1, 4) # 1, 2, or 3
    bob_basis_choice = np.random.randint(1, 4)   # 1, 2, or 3

    # 2. Create Bell Pair |Psi->
    qc, qr, cr = create_bell_psi_minus()

    # 3. Apply measurement rotations based on choices
    apply_measurement_rotation(qc, qr[0], alice_angles[alice_basis_choice])
    apply_measurement_rotation(qc, qr[1], bob_angles[bob_basis_choice])
    qc.barrier() # Visual separator

    # 4. Measure in the computational (Z) basis
    # Maps qr[0] -> cr[0] (Alice), qr[1] -> cr[1] (Bob)
    qc.measure(qr, cr)

    # 5. Transpile for the simulator
    tqc = transpile(qc, simulator)

    # 6. Run the simulation (1 shot per loop iteration)
    # Using run() directly is fine for shot-by-shot simulation like this.
    job = simulator.run(tqc, shots=1, memory=True) # Use memory=True to get individual shot results
    result = job.result()
    memory = result.get_memory(tqc) # Returns list like ['0 1']

    if memory:
        measurement_outcome = memory[0] # Get the single shot result 'b a' (Bob Alice)
        # Check if the string has the expected length (NUM_QUBITS)
        if len(measurement_outcome) == NUM_QUBITS:
             # Assuming format 'ba' ('<cr[1]_result><cr[0]_result>')
             # Bob's result (cr[1]) is at index 0
             # Alice's result (cr[0]) is at index 1
            bob_outcome = measurement_outcome[0]
            alice_outcome = measurement_outcome[1]


            # 7. Store results
            results.append({
                'alice_basis': alice_basis_choice,
                'bob_basis': bob_basis_choice,
                'alice_outcome': alice_outcome,     # Storing Alice's outcome '0' or '1'
                'bob_outcome': bob_outcome          # Storing Bob's outcome '0' or '1'
                })
        else:
             # Handle unexpected format
            print(f"Warning: Shot {i+1}: Unexpected measurement outcome format: '{measurement_outcome}'. Skipping.")
             # Optionally add more robust error handling or debugging here
    else:
        print(f"Warning: Shot {i+1}: No memory result returned from simulator.")


    # Optional: Print progress
    if (i + 1) % (N_SHOTS // 10) == 0:
        print(f"  Completed {i + 1}/{N_SHOTS} rounds...")

print("Simulation loop finished.")
print(f"Total results collected: {len(results)}")

# Display first few results for verification
print("\nSample results (first 5):")
for k in range(min(5, len(results))):
    print(results[k])


Starting simulation loop...
  Completed 200/2000 rounds...
  Completed 400/2000 rounds...
  Completed 600/2000 rounds...
  Completed 800/2000 rounds...
  Completed 1000/2000 rounds...
  Completed 1200/2000 rounds...
  Completed 1400/2000 rounds...
  Completed 1600/2000 rounds...
  Completed 1800/2000 rounds...
  Completed 2000/2000 rounds...
Simulation loop finished.
Total results collected: 2000

Sample results (first 5):
{'alice_basis': 1, 'bob_basis': 2, 'alice_outcome': '1', 'bob_outcome': '1'}
{'alice_basis': 2, 'bob_basis': 2, 'alice_outcome': '0', 'bob_outcome': '1'}
{'alice_basis': 3, 'bob_basis': 1, 'alice_outcome': '0', 'bob_outcome': '0'}
{'alice_basis': 1, 'bob_basis': 2, 'alice_outcome': '1', 'bob_outcome': '0'}
{'alice_basis': 3, 'bob_basis': 2, 'alice_outcome': '0', 'bob_outcome': '1'}


## 4. Classical Post-Processing and CHSH Test

Now, Alice and Bob publicly share their basis choices and perform the CHSH test using the agreed-upon basis pairs.

In [10]:
# --- Data Grouping ---
grouped_results = {} # Dictionary to store results grouped by (alice_basis, bob_basis)
for res in results:
    basis_pair = (res['alice_basis'], res['bob_basis'])
    if basis_pair not in grouped_results:
        grouped_results[basis_pair] = []
    grouped_results[basis_pair].append((res['alice_outcome'], res['bob_outcome']))

print("\n--- Basis Reconciliation and Data Grouping ---")
for basis_pair, outcomes in grouped_results.items():
    print(f"Basis Pair (A{basis_pair[0]}, B{basis_pair[1]}): {len(outcomes)} results")


--- Basis Reconciliation and Data Grouping ---
Basis Pair (A1, B2): 224 results
Basis Pair (A2, B2): 221 results
Basis Pair (A3, B1): 221 results
Basis Pair (A3, B2): 213 results
Basis Pair (A1, B1): 234 results
Basis Pair (A2, B1): 234 results
Basis Pair (A1, B3): 214 results
Basis Pair (A2, B3): 215 results
Basis Pair (A3, B3): 224 results


In [11]:
# --- CHSH Correlation Calculation ---
# E(a, b) = <A_a * B_b> = Avg[ value(alice_outcome) * value(bob_outcome) ]
# where outcomes {0, 1} map to values {+1, -1}
correlation_values = {}
print("\n--- Calculating Correlation Values E(a, b) ---")

for basis_pair in CHSH_BASIS_PAIRS:
    if basis_pair in grouped_results:
        outcomes = grouped_results[basis_pair]
        if not outcomes:
            correlation_values[basis_pair] = 0 # Handle case with no results for this pair
            print(f"Basis Pair {basis_pair}: No results found.")
            continue

        sum_products = 0
        for alice_outcome, bob_outcome in outcomes:
            alice_val = outcome_to_value(alice_outcome)
            bob_val = outcome_to_value(bob_outcome)
            sum_products += alice_val * bob_val

        correlation = sum_products / len(outcomes)
        correlation_values[basis_pair] = correlation
        print(f"E(A{basis_pair[0]}, B{basis_pair[1]}) = {correlation:.4f} ({len(outcomes)} samples)")
    else:
        correlation_values[basis_pair] = 0 # Or handle as error/insufficient data
        print(f"Basis Pair {basis_pair}: No results found.")


# --- Calculate CHSH Value S ---
# S = E(A1, B1) - E(A1, B3) + E(A3, B1) + E(A3, B3) (Ekert's formulation)
S_value = (correlation_values.get((1, 1), 0) -
           correlation_values.get((1, 3), 0) +
           correlation_values.get((3, 1), 0) +
           correlation_values.get((3, 3), 0))

print("\n--- CHSH Test ---")
print(f"Calculated S value: {S_value:.4f}")


--- Calculating Correlation Values E(a, b) ---
E(A1, B1) = -0.6325 (234 samples)
E(A1, B3) = 0.7290 (214 samples)
E(A3, B1) = -0.7466 (221 samples)
E(A3, B3) = -0.6875 (224 samples)

--- CHSH Test ---
Calculated S value: -2.7956


In [12]:
# Theoretical maximum for |Psi-> state and these angles: -2*sqrt(2)
theoretical_S = -2 * math.sqrt(2)
print(f"Theoretical S value for |Psi->: {theoretical_S:.4f}")
print(f"Classical limit: |S| <= 2")

# Check for security
SECURITY_THRESHOLD = 2.1 # A value significantly above 2
if abs(S_value) > SECURITY_THRESHOLD:
    print(f"Test PASSED: |S| = {abs(S_value):.4f} > {SECURITY_THRESHOLD}. Minimal eavesdropping detected.")
    secure = True
else:
    print(f"Test FAILED: |S| = {abs(S_value):.4f} <= {SECURITY_THRESHOLD}. Potential eavesdropping or high noise!")
    secure = False


Theoretical S value for |Psi->: -2.8284
Classical limit: |S| <= 2
Test PASSED: |S| = 2.7956 > 2.1. Minimal eavesdropping detected.



## 5. Key Generation

If the CHSH test passes, Alice and Bob can generate a raw key using measurement results from basis pairs *not* used in the CHSH calculation. We need to identify pairs with strong correlation or anti-correlation.

For $|\Psi^-\rangle$, the theoretical correlation is $E(\theta_A, \theta_B) = -\cos(\theta_A - \theta_B)$.
* If $E = -1$ (e.g., $\theta_A = \theta_B$, like $A_2, B_1$), outcomes are perfectly anti-correlated ($01$ or $10$). Bob flips his bit.
* If $E = +1$ (e.g., $\theta_A - \theta_B = \pi$), outcomes are perfectly correlated ($00$ or $11$). Bob keeps his bit.
* Other pairs might have partial correlation. Typically, only strongly correlated/anti-correlated pairs are used for the key.

Let's check correlation for a potential key-generating pair, e.g., $(A_2, B_1)$ where $\theta_A = \pi/4, \theta_B = \pi/4$. $E = -\cos(0) = -1$. (Anti-correlated)
Another pair: $(A_2, B_2)$ where $\theta_A = \pi/4, \theta_B = \pi/2$. $E = -\cos(-\pi/4) = -1/\sqrt{2} \approx -0.707$. (Partially anti-correlated)
Let's use $(A_2, B_1)$ for the key, expecting anti-correlation.

In [13]:
print("\n--- Raw Key Generation ---")
if secure:
    # Use basis pair (A2, B1) for key generation
    key_basis_pair = (2, 1)
    alice_raw_key = []
    bob_raw_key = []
    mismatches = 0

    if key_basis_pair in grouped_results:
        key_outcomes = grouped_results[key_basis_pair]
        print(f"Using basis pair {key_basis_pair} for key generation ({len(key_outcomes)} results).")
        print("Expecting anti-correlation (E = -1). Bob should flip his bits.")

        for alice_outcome, bob_outcome in key_outcomes:
            alice_bit = int(alice_outcome)
            bob_bit = int(bob_outcome)

            # Bob flips his bit due to expected anti-correlation
            bob_final_bit = 1 - bob_bit

            alice_raw_key.append(alice_bit)
            bob_raw_key.append(bob_final_bit)

            if alice_bit != bob_final_bit:
                mismatches += 1

        print(f"Generated raw key length: {len(alice_raw_key)}")
        if len(alice_raw_key) > 0:
            qber = mismatches / len(alice_raw_key)
            print(f"Quantum Bit Error Rate (QBER) estimate: {qber:.4f}")
            # In ideal simulation, QBER should be 0.

            # Display a sample of the key
            sample_len = min(len(alice_raw_key), 20)
            print(f"Alice's raw key sample: {''.join(map(str, alice_raw_key[:sample_len]))}")
            print(f"Bob's raw key sample:   {''.join(map(str, bob_raw_key[:sample_len]))}")
        else:
            print("No results found for the chosen key generation basis pair.")

    else:
        print(f"Basis pair {key_basis_pair} not found in results. Cannot generate key.")

else:
    print("CHSH test failed. No key generated due to security concerns.")




--- Raw Key Generation ---
Using basis pair (2, 1) for key generation (234 results).
Expecting anti-correlation (E = -1). Bob should flip his bits.
Generated raw key length: 234
Quantum Bit Error Rate (QBER) estimate: 0.0000
Alice's raw key sample: 10101100000011011111
Bob's raw key sample:   10101100000011011111




## 6. Conclusion

This notebook simulated the E91 protocol. We generated entangled pairs, simulated measurements in randomly chosen bases, performed the CHSH test to check for eavesdropping, and extracted a raw secret key if the test passed.

In this ideal, noise-free simulation:
* The CHSH value $S$ should be close to the theoretical maximum of $|-2\sqrt{2}| \approx 2.828$.
* The QBER for the generated key (using appropriate basis pairs and bit flipping) should be close to 0.

Deviations from these values in a real experiment or noisy simulation would indicate errors or potential eavesdropping, requiring further steps like error correction and privacy amplification.